In [3]:
import axelrod as axl
import networkx as nx
import matplotlib.pyplot as plt
from scipy.io import mmread
import numpy as np
import pandas as pd

Base Algorithm

In [ ]:
file_path = "fb_graph/matname.mtx"
sparse_matrix = mmread(file_path)
graph = nx.Graph(sparse_matrix)

strategies = [
    axl.Cooperator(),
    axl.Defector(),
    axl.TitForTat(),
    axl.Grudger(),
    axl.Random(),
    axl.ZDExtortion(),
    axl.Capri(),
    axl.Appeaser()
]

players = []
for node in graph.nodes:
    players.append(np.random.choice(strategies))

# turns is ther number of iterations between each pair
spatial_tournament = axl.Tournament(
    players, edges=graph.edges, turns=100, repetitions=1)
results = spatial_tournament.play(filename="test.csv")

df = pd.read_csv("test.csv")


In [5]:
df = pd.read_csv("100_turns.csv")

print(df.columns)

Index(['Interaction index', 'Player index', 'Opponent index', 'Repetition',
       'Player name', 'Opponent name', 'Actions', 'Score', 'Score difference',
       'Turns', 'Score per turn', 'Score difference per turn', 'Win',
       'Initial cooperation', 'Cooperation count', 'CC count', 'CD count',
       'DC count', 'DD count', 'CC to C count', 'CC to D count',
       'CD to C count', 'CD to D count', 'DC to C count', 'DC to D count',
       'DD to C count', 'DD to D count', 'Good partner'],
      dtype='object')


Getting best strageties for X turns.

In [11]:
import pandas as pd

# Load results from the tournament
df = pd.read_csv("100_turns.csv")

# Compute total score for each player
scores = df.groupby("Player index")["Score"].sum()

# Extract unique strategies based on the "Player index" and "Player name"
strategy_mapping = df[["Player index", "Player name"]
                      ].drop_duplicates().set_index("Player index")["Player name"]

# Add strategy information to the scores DataFrame
scores_with_strategy = pd.DataFrame(scores).reset_index()
scores_with_strategy["Strategy"] = scores_with_strategy["Player index"].map(
    strategy_mapping)

# Compute mean score by strategy
strategy_scores = scores_with_strategy.groupby("Strategy")["Score"].mean()

# Calculate the overall mean score
overall_mean_score = strategy_scores.mean()

# Categorize strategies as good or bad based on their mean score
good_strategies = strategy_scores[strategy_scores > overall_mean_score]
bad_strategies = strategy_scores[strategy_scores <= overall_mean_score]

# Output the results
print("Good Strategies:")
for strategy, mean_score in good_strategies.items():
    print(f"Strategy: {strategy} - Mean Score: {mean_score:.2f}")

print("\nBad Strategies:")
for strategy, mean_score in bad_strategies.items():
    print(f"Strategy: {strategy} - Mean Score: {mean_score:.2f}")

Good Strategies:
Strategy: Appeaser - Mean Score: 15469.33
Strategy: CAPRI - Mean Score: 16736.94
Strategy: Cooperator - Mean Score: 15437.00
Strategy: Grudger - Mean Score: 18450.86
Strategy: Tit For Tat - Mean Score: 16543.53

Bad Strategies:
Strategy: Defector - Mean Score: 14075.96
Strategy: Random: 0.5 - Mean Score: 11875.78
Strategy: ZD-Extortion: 0.2, 0.1, 1 - Mean Score: 12092.30


Checks what is better, playing as Defector or Cooperator

In [14]:
import pandas as pd

# Load results from the tournament
df = pd.read_csv("100_turns.csv")

# Separate games based on the player's strategy
cooperator_games = df[df["Player name"] == "Cooperator"]
defector_games = df[df["Player name"] == "Defector"]

# Calculate mean and standard deviation for Cooperators
cooperator_avg_score = cooperator_games["Score"].mean()
cooperator_std_dev = cooperator_games["Score"].std()

# Calculate mean and standard deviation for Defectors
defector_avg_score = defector_games["Score"].mean()
defector_std_dev = defector_games["Score"].std()

# Analyze performance based on opponent's strategy
cooperator_vs_opponents = cooperator_games.groupby(
    "Opponent name")["Score"].agg(["mean", "std"])
defector_vs_opponents = defector_games.groupby(
    "Opponent name")["Score"].agg(["mean", "std"])

# Output the results
print(f"Average Score for Cooperators: {
      cooperator_avg_score:.2f} (Std Dev: {cooperator_std_dev:.2f})")
print(f"Average Score for Defectors: {
      defector_avg_score:.2f} (Std Dev: {defector_std_dev:.2f})")

if cooperator_avg_score > defector_avg_score:
    print("It is better to play as a Cooperator.")
elif defector_avg_score > cooperator_avg_score:
    print("It is better to play as a Defector.")
else:
    print("Playing as a Cooperator or Defector is equally effective.")

print("\nPerformance of Cooperators Against Opponents:")
print(cooperator_vs_opponents)

print("\nPerformance of Defectors Against Opponents:")
print(defector_vs_opponents)

Average Score for Cooperators: 222.88 (Std Dev: 107.67)
Average Score for Defectors: 202.53 (Std Dev: 141.99)
It is better to play as a Cooperator.

Performance of Cooperators Against Opponents:
                                 mean        std
Opponent name                                   
Appeaser                   300.000000   0.000000
CAPRI                      300.000000   0.000000
Cooperator                 300.000000   0.000000
Defector                     0.000000   0.000000
Grudger                    300.000000   0.000000
Random: 0.5                150.228571  15.106975
Tit For Tat                300.000000   0.000000
ZD-Extortion: 0.2, 0.1, 1  134.112330  21.726942

Performance of Defectors Against Opponents:
                                 mean        std
Opponent name                                   
Appeaser                   300.000000   0.000000
CAPRI                      104.000000   0.000000
Cooperator                 500.000000   0.000000
Defector                 

In [16]:
import pandas as pd
import networkx as nx
from scipy.io import mmread

# Load results from the tournament
df = pd.read_csv("100_turns.csv")

# Load the graph to compute clustering coefficients
file_path = "fb_graph/matname.mtx"  # Path to your graph file
sparse_matrix = mmread(file_path)
graph = nx.Graph(sparse_matrix)

# Compute clustering coefficients for all nodes
clustering_coeffs = nx.clustering(graph)
clustering_df = pd.DataFrame(list(clustering_coeffs.items()), columns=[
                             "Player index", "Clustering Coefficient"])

# Merge clustering coefficients with player scores
scores = df.groupby("Player index")["Score"].sum()
strategy_mapping = df[["Player index", "Player name"]
                      ].drop_duplicates().set_index("Player index")["Player name"]
scores_with_clustering = pd.DataFrame(scores).reset_index()
scores_with_clustering["Strategy"] = scores_with_clustering["Player index"].map(
    strategy_mapping)
scores_with_clustering = scores_with_clustering.merge(
    clustering_df, on="Player index")

# Calculate average score for each strategy
strategy_stats = scores_with_clustering.groupby(
    "Strategy")["Score"].agg(["mean"]).reset_index()

# Find strategies with highest/lowest average scores
highest_mean_strategy = strategy_stats.loc[strategy_stats["mean"].idxmax()]
lowest_mean_strategy = strategy_stats.loc[strategy_stats["mean"].idxmin()]

# Categorize nodes into high and low clustering coefficient groups
median_clustering = scores_with_clustering["Clustering Coefficient"].median()
high_cluster_nodes = scores_with_clustering[scores_with_clustering["Clustering Coefficient"] > median_clustering]
low_cluster_nodes = scores_with_clustering[scores_with_clustering["Clustering Coefficient"]
                                           <= median_clustering]

# Calculate average score for strategies among high-cluster-coefficient nodes
high_cluster_stats = high_cluster_nodes.groupby(
    "Strategy")["Score"].agg(["mean"]).reset_index()
highest_mean_high_cluster = high_cluster_stats.loc[high_cluster_stats["mean"].idxmax(
)]
lowest_mean_high_cluster = high_cluster_stats.loc[high_cluster_stats["mean"].idxmin(
)]

# Calculate average score for strategies among low-cluster-coefficient nodes
low_cluster_stats = low_cluster_nodes.groupby(
    "Strategy")["Score"].agg(["mean"]).reset_index()
highest_mean_low_cluster = low_cluster_stats.loc[low_cluster_stats["mean"].idxmax(
)]
lowest_mean_low_cluster = low_cluster_stats.loc[low_cluster_stats["mean"].idxmin(
)]

# Output results
print("Highest Average Score:")
print(f"Strategy: {highest_mean_strategy['Strategy']
                   } - Mean Score: {highest_mean_strategy['mean']:.2f}")

print("\nLowest Average Score:")
print(f"Strategy: {lowest_mean_strategy['Strategy']
                   } - Mean Score: {lowest_mean_strategy['mean']:.2f}")

print("\nHighest Average Score Among High Cluster-Coefficient Nodes:")
print(f"Strategy: {highest_mean_high_cluster['Strategy']
                   } - Mean Score: {highest_mean_high_cluster['mean']:.2f}")

print("\nLowest Average Score Among High Cluster-Coefficient Nodes:")
print(f"Strategy: {lowest_mean_high_cluster['Strategy']
                   } - Mean Score: {lowest_mean_high_cluster['mean']:.2f}")

print("\nHighest Average Score Among Low Cluster-Coefficient Nodes:")
print(f"Strategy: {highest_mean_low_cluster['Strategy']
                   } - Mean Score: {highest_mean_low_cluster['mean']:.2f}")

print("\nLowest Average Score Among Low Cluster-Coefficient Nodes:")
print(f"Strategy: {lowest_mean_low_cluster['Strategy']
                   } - Mean Score: {lowest_mean_low_cluster['mean']:.2f}")

Highest Average Score:
Strategy: Grudger - Mean Score: 18450.86

Lowest Average Score:
Strategy: Random: 0.5 - Mean Score: 11875.78

Highest Average Score Among High Cluster-Coefficient Nodes:
Strategy: Grudger - Mean Score: 13814.28

Lowest Average Score Among High Cluster-Coefficient Nodes:
Strategy: Random: 0.5 - Mean Score: 8712.89

Highest Average Score Among Low Cluster-Coefficient Nodes:
Strategy: Grudger - Mean Score: 23181.11

Lowest Average Score Among Low Cluster-Coefficient Nodes:
Strategy: Random: 0.5 - Mean Score: 14890.87


Calculating sd for every strategy

In [2]:
import pandas as pd

# Load results from the tournament
df = pd.read_csv("100_turns.csv")

# Compute total score for each player
scores = df.groupby("Player index")["Score"].sum()

# Extract unique strategies based on the "Player index" and "Player name"
strategy_mapping = df[["Player index", "Player name"]
                      ].drop_duplicates().set_index("Player index")["Player name"]

# Add strategy information to the scores DataFrame
scores_with_strategy = pd.DataFrame(scores).reset_index()
scores_with_strategy["Strategy"] = scores_with_strategy["Player index"].map(
    strategy_mapping)

# Compute mean and standard deviation scores by strategy
strategy_stats = scores_with_strategy.groupby(
    "Strategy")["Score"].agg(["mean", "std"]).reset_index()

# Normalize the standard deviation
std_min = strategy_stats["std"].min()
std_max = strategy_stats["std"].max()
epsilon = 1e-8  # Small constant to avoid division by zero

strategy_stats["Normalized Std"] = (
    strategy_stats["std"] - std_min) / (std_max - std_min + epsilon)

# Output the results
print("Strategy Stats with Normalized Standard Deviation:")
print(strategy_stats)

# Separate good and bad strategies based on normalized mean score
overall_mean_score = strategy_stats["mean"].mean()
good_strategies = strategy_stats[strategy_stats["mean"] > overall_mean_score]
bad_strategies = strategy_stats[strategy_stats["mean"] <= overall_mean_score]

print("\nGood Strategies:")
for _, row in good_strategies.iterrows():
    print(f"Strategy: {row['Strategy']} - Mean Score: {row['mean']          :.2f} - Normalized Std: {row['Normalized Std']:.2f}")

print("\nBad Strategies:")
for _, row in bad_strategies.iterrows():
    print(f"Strategy: {row['Strategy']} - Mean Score: {row['mean']          :.2f} - Normalized Std: {row['Normalized Std']:.2f}")

Strategy Stats with Normalized Standard Deviation:
                    Strategy          mean           std  Normalized Std
0                   Appeaser  15469.325130  14070.697624        0.633101
1                      CAPRI  16736.940431  14648.673434        0.729308
2                 Cooperator  15437.002472  14131.982741        0.643303
3                   Defector  14075.962914  12672.643093        0.400390
4                    Grudger  18450.861250  16084.529397        0.968311
5                Random: 0.5  11875.777512  10267.230558        0.000000
6                Tit For Tat  16543.525292  16274.905026        1.000000
7  ZD-Extortion: 0.2, 0.1, 1  12092.302108  11657.093771        0.231348

Good Strategies:
Strategy: Appeaser - Mean Score: 15469.33 - Normalized Std: 0.63
Strategy: CAPRI - Mean Score: 16736.94 - Normalized Std: 0.73
Strategy: Cooperator - Mean Score: 15437.00 - Normalized Std: 0.64
Strategy: Grudger - Mean Score: 18450.86 - Normalized Std: 0.97
Strategy: Tit Fo

For ranking of nodes and relations cutting and relations making

In [4]:
import random
import networkx as nx
import matplotlib.pyplot as plt
import math
import csv

from scipy.io import mmread
from collections import defaultdict
from tqdm import tqdm


# Define all strategies (including new ones)
STRATEGIES = [
    "TitForTat",
    "AlwaysDefect",
    "AlwaysCooperate",
    "Grudger",
    "Pavlovian",
    "Appeaser",
    "Capri"
]

# Payoff matrix and helper functions
PAYOFF_MATRIX = {
    ("C", "C"): (3, 3),
    ("C", "D"): (0, 5),
    ("D", "C"): (5, 0),
    ("D", "D"): (1, 1),
}


class Agent:
    def __init__(self, name):
        self.name = name
        self.strategy = random.choice(STRATEGIES)  # Random initial strategy
        self.strategy_history = []  # Track strategy at the start of each iteration
        self.average_payoff_history = []  # Average payoff at the end of each iteration
        # {opponent_name: [(my_action, opp_action)]}
        self.memory = defaultdict(list)
        self.score = 0
        self.reputation = 1.0
        self.defection_count = defaultdict(int)
        self.interaction_count = 0  # Track total interactions

    def decide(self, opponent):
        # First interaction: use opponent's reputation
        if opponent.name not in self.memory:
            return "C" if random.random() < opponent.reputation else "D"

        # Strategy-specific logic
        if self.strategy == "TitForTat":
            return self.memory[opponent.name][-1][1] if self.memory[opponent.name] else "C"
        elif self.strategy == "AlwaysDefect":
            return "D"
        elif self.strategy == "AlwaysCooperate":
            return "C"
        elif self.strategy == "Grudger":
            return self._decide_grudger(opponent)
        elif self.strategy == "Pavlovian":
            return self._decide_pavlovian(opponent)
        elif self.strategy == "Appeaser":
            return self._decide_appeaser(opponent)
        elif self.strategy == "Capri":
            return self._decide_capri(opponent)
        else:
            return "C"  # Fallback

    def _decide_grudger(self, opponent):
        history = self.memory[opponent.name]
        if any(opp_action == "D" for (_, opp_action) in history):
            return "D"
        return "C"

    def _decide_pavlovian(self, opponent):
        last_my_action, last_opp_action = self.memory[opponent.name][-1]
        if last_my_action == last_opp_action:
            return last_my_action  # Win-Stay
        return "D" if last_my_action == "C" else "C"  # Lose-Shift

    def _decide_appeaser(self, opponent):
        current_action = "C"
        for _, opp_action in self.memory[opponent.name]:
            if opp_action == "D":
                current_action = "D" if current_action == "C" else "C"
        return current_action

    def _decide_capri(self, opponent):
        history = self.memory.get(opponent.name, [])
        if len(history) < 3:
            return "C" if random.random() < opponent.reputation else "D"

        # Extract last 3 moves for both players
        last3 = history[-3:]
        last3_me = [h[0] for h in last3]  # Our last 3 actions (oldest first)
        last3_opp = [h[1] for h in last3]  # Opponent's last 3 actions

        # Rule C: Mutual cooperation
        if last3_me == ["C", "C", "C"] and last3_opp == ["C", "C", "C"]:
            return "C"

        # Rule A: Accept punishment cases
        a_cases = [
            (["C", "C", "D"], ["C", "C", "C"]),  # (ccd, ccc)
            (["C", "D", "C"], ["C", "C", "D"]),  # (cdc, ccd)
            (["D", "C", "C"], ["C", "D", "C"]),  # (dcc, cdc)
            (["C", "C", "C"], ["D", "C", "C"])   # (ccc, dcc)
        ]
        if (last3_me, last3_opp) in a_cases:
            return "C"

        # Rule P: Punish and follow-up cases
        p_punish_case = (["C", "C", "C"], ["C", "C", "D"])  # (ccc, ccd)
        p_cooperate_cases = [
            (["C", "C", "D"], ["C", "D", "C"]),  # (ccd, cdc)
            (["C", "D", "C"], ["D", "C", "C"]),  # (cdc, dcc)
            (["D", "C", "C"], ["C", "C", "C"])   # (dcc, ccc)
        ]
        if (last3_me, last3_opp) == p_punish_case:
            return "D"
        elif (last3_me, last3_opp) in p_cooperate_cases:
            return "C"

        # Rule R: Recovery cases
        r_cases = [
            (["D", "D", "D"], ["D", "D", "C"]),  # (ddd, ddc)
            (["D", "D", "C"], ["D", "C", "C"]),  # (ddc, dcc)
            (["D", "C", "C"], ["C", "C", "C"]),  # (dcc, ccc)
            (["D", "D", "C"], ["D", "D", "D"]),  # (ddc, ddd)
            (["D", "C", "C"], ["D", "D", "C"]),  # (dcc, ddc)
            (["C", "C", "C"], ["D", "C", "C"]),  # (ccc, dcc)
            (["D", "D", "C"], ["D", "D", "C"]),  # (ddc, ddc)
            (["D", "C", "C"], ["D", "C", "C"])   # (dcc, dcc)
        ]
        if (last3_me, last3_opp) in r_cases:
            return "C"

        # Rule I: Default to defect
        return "D"


def update_reputation(agent, action, num_friends):
    """
    - If we cooperate then our reputation increases.
    - If we deflect then our reputation decreases.
    - It increases/decreases more for agents with less friends.
    """
    scale = math.exp(-0.05 * num_friends)
    if action == "C":
        agent.reputation = min(1.0, agent.reputation + 0.1 * scale)
    else:
        agent.reputation = max(0.0, agent.reputation - 0.1 * scale)


def adapt_strategy(agent, neighbors, agents, current_iteration):
    """
    - We don't do strategy adoptions in first 10 rounds.
    - After that, we adopt the strategy of neighbour with highest average payoff
    with probability increasing with difference in our average payoffs.
    """
    if current_iteration < 10:
        return

    best_neighbor = None
    best_avg = -float('inf')
    my_avg = agent.score / (agent.interaction_count + 1e-6)

    for neighbor in neighbors:
        neighbor_agent = agents[neighbor]
        if neighbor_agent.interaction_count == 0:
            continue
        neighbor_avg = neighbor_agent.score / neighbor_agent.interaction_count
        if neighbor_avg > best_avg and neighbor_avg > my_avg:
            best_avg = neighbor_avg
            best_neighbor = neighbor

    if best_neighbor:
        # Probability proportional to payoff difference
        payoff_diff = best_avg - my_avg
        if payoff_diff > 0:
            prob = min(1.0, payoff_diff / (my_avg + 1e-6))
            if random.random() < prob:
                agent.strategy = agents[best_neighbor].strategy


def simulate(
    graph,
    iterations=100,
    interaction_csv="interaction.csv",
    strategy_csv="strategy_history.csv",
    payoff_csv="average_payoff.csv"
):
    """
    At each iteration, each agent plays one turn with each of its neighbours.
    """
    agents = {node: Agent(node) for node in graph.nodes()}
    # {agent: {opponent: "CCDCD..."}}
    action_history = defaultdict(lambda: defaultdict(str))

    for iteration in tqdm(range(iterations)):
        # Record strategy at the **start** of the iteration
        for agent in agents.values():
            agent.strategy_history.append(agent.strategy)

        current_edges = list(graph.edges())
        for edge in current_edges:
            agent1, agent2 = agents[edge[0]], agents[edge[1]]

            action1 = agent1.decide(agent2)
            action2 = agent1.decide(agent2)

            # Update scores and memory
            payoff1, payoff2 = PAYOFF_MATRIX[(action1, action2)]
            agent1.score += payoff1
            agent2.score += payoff2
            agent1.interaction_count += 1
            agent2.interaction_count += 1

            agent1.memory[agent2.name].append((action1, action2))
            agent2.memory[agent1.name].append((action2, action1))

            if action1 == "D":
                agent1.defection_count[agent2.name] += 1
            if action2 == "D":
                agent2.defection_count[agent1.name] += 1

            # Update reputations with friend-count scaling
            num_friends_agent1 = len(list(graph.neighbors(agent1.name)))
            num_friends_agent2 = len(list(graph.neighbors(agent2.name)))
            update_reputation(agent1, action1, num_friends_agent1)
            update_reputation(agent2, action2, num_friends_agent2)

            # Log actions to history
            action_history[agent1.name][agent2.name] += action1
            action_history[agent2.name][agent1.name] += action2

        # Calculate average payoff for this iteration and append to history
        for agent in agents.values():
            if agent.interaction_count == 0:
                avg = 0.0
            else:
                avg = agent.score / agent.interaction_count
            agent.average_payoff_history.append(round(avg, 3))

        # Update strategies (after iteration 10)
        links_to_remove = []
        links_to_add = []
        for node in graph.nodes():
            agent = agents[node]
            neighbors = list(graph.neighbors(node))

            # Possibly adapt strategy to best neighbor
            adapt_strategy(agent, neighbors, agents, iteration)

            # Break links based on defections
            for neighbor in neighbors:
                defections = agent.defection_count.get(neighbor, 0)
                if random.random() < defections / 10.0:
                    links_to_remove.append((agent.name, neighbor))

            # Create links (preferential attachment)
            current_friends = list(graph.neighbors(agent.name))
            num_friends = len(current_friends)
            prob_create = num_friends / (graph.number_of_nodes() + 1e-6)
            if random.random() < prob_create:
                non_friends = [n for n in graph.nodes(
                ) if n != agent.name and not graph.has_edge(agent.name, n)]
                if non_friends:
                    new_friend = random.choice(non_friends)
                    links_to_add.append((agent.name, new_friend))

        # Apply graph changes
        for link in links_to_remove:
            if graph.has_edge(*link):
                graph.remove_edge(*link)
        for link in links_to_add:
            if not graph.has_edge(*link):
                graph.add_edge(*link)

    print("Saving to csv file...")
    # Save interaction history
    with open(interaction_csv, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for agent in action_history:
            for opponent in action_history[agent]:
                writer.writerow(
                    [agent, opponent, action_history[agent][opponent]])

    # Save strategy history
    with open(strategy_csv, "w", newline="") as f:
        writer = csv.writer(f)
        header = ["Node"] + [str(i) for i in range(iterations)]
        writer.writerow(header)
        for node in agents:
            row = [node] + agents[node].strategy_history
            writer.writerow(row)

    # Save average payoff history
    with open(payoff_csv, "w", newline="") as f:
        writer = csv.writer(f)
        header = ["Player"] + [str(i) for i in range(iterations)]
        writer.writerow(header)
        for node in agents:
            row = [node] + agents[node].average_payoff_history
            writer.writerow(row)

    return agents, graph


if __name__ == "__main__":
    print("reading graph..")
    mtx_file = "fb_graph/matname.mtx"
    sparse_matrix = mmread(mtx_file)
    print("finished reading graph")
    G = nx.Graph(sparse_matrix)

    n_iter = 100
    agents, final_graph = simulate(
        G,
        iterations=n_iter,
        interaction_csv=f"interactions_{n_iter}.csv",
        strategy_csv=f"strategy_history_{n_iter}.csv",
        payoff_csv=f"avg_payoff_history_{n_iter}.csv"
    )

reading graph..
finished reading graph


100%|██████████| 100/100 [01:28<00:00,  1.13it/s]


Saving to csv file...
